In [1]:
import numpy as np
import pandas as pd
import scipy
from sklearn.decomposition import TruncatedSVD

In [2]:
task = "cite"

In [2]:
task = "multi"

In [3]:
if task == "cite":
    train_X_values = scipy.sparse.load_npz(
        "../../data/input/compressed/train_cite_inputs_values.sparse.npz"
    )
    train_X_idxcol = np.load(
        "../../data/input/compressed/train_cite_inputs_idxcol.npz", allow_pickle=True
    )
    test_X_values = scipy.sparse.load_npz(
        "../../data/input/compressed/test_cite_inputs_values.sparse.npz"
    )
    test_X_idxcol = np.load(
        "../../data/input/compressed/test_cite_inputs_idxcol.npz", allow_pickle=True
    )
    test_X_2_values = pd.read_hdf("../../data/input/test_cite_inputs_day_2_donor_27678.h5").values
    test_X_2_columns = pd.read_hdf("../../data/input/test_cite_inputs_day_2_donor_27678.h5").columns
else:
    train_X_values = scipy.sparse.load_npz(
        "../../data/input/compressed/train_multi_inputs_values.sparse.npz"
    )
    train_X_idxcol = np.load(
        "../../data/input/compressed/train_multi_inputs_idxcol.npz", allow_pickle=True
    )
    test_X_values = scipy.sparse.load_npz(
        "../../data/input/compressed/test_multi_inputs_values.sparse.npz"
    )
    test_X_idxcol = np.load(
        "../../data/input/compressed/test_multi_inputs_idxcol.npz", allow_pickle=True
    )

Correlation

In [4]:
if task == "cite":
    correlations = np.load("../../data/input/correlation/correlations_citeseq.npz")["correlations"]
    np_inputs_names = np.load("../../data/input/compressed/train_cite_inputs_idxcol.npz", allow_pickle=True)["columns"]
    np_targets_names = np.load("../../data/input/compressed/train_cite_targets_idxcol.npz", allow_pickle=True)["columns"]
else:
    correlations = scipy.sparse.load_npz("../../data/input/correlation/correlations_multiome.sparse.npz")
    np_inputs_names = np.load("../../data/input/compressed/train_multi_inputs_idxcol.npz", allow_pickle=True)["columns"]
    np_targets_names = np.load("../../data/input/compressed/train_multi_targets_idxcol.npz", allow_pickle=True)["columns"]
    correlations = correlations.tocsc()

In [5]:
#most_importants_inputs = np.argsort(np.squeeze(np.asarray(np.mean(np.abs(correlations), axis=0))))[::-1]
most_importants_inputs = np.argsort(np.squeeze(np.asarray(np.mean(np.abs(correlations), axis=0))))[::-1]

name_col = []
mean_abs_cor_col = []
mean_cor_col = []
pc_positive_cor_col = []
pc_negative_cor_col = []
min_col = []
max_col = []

for num_input in most_importants_inputs:
    mean_abs_cor = np.mean(np.abs(correlations[:,num_input]))
    mean_cor = np.mean(correlations[:,num_input])
    pc_positive_cor = np.mean(correlations[:,num_input]>0.01)*100
    pc_negative_cor = np.mean(correlations[:,num_input]<-0.01)*100
    min_val = np.min(correlations[:,num_input])
    max_val = np.max(correlations[:,num_input])

    name_col.append(np_inputs_names[num_input])
    mean_abs_cor_col.append(mean_abs_cor)
    mean_cor_col.append(mean_cor)
    pc_positive_cor_col.append(pc_positive_cor)
    pc_negative_cor_col.append(pc_negative_cor)

    min_col.append(min_val)
    max_col.append(max_val)

df = pd.DataFrame({"Input":name_col,
            "avg abs cor": mean_abs_cor_col,
            "avg cor": mean_cor_col,
            "%positive": pc_positive_cor_col,
            "%negative": pc_negative_cor_col,
            "min":min_col,
            "max":max_col
            } )

In [6]:
cite_abg_abs_cor_10 = list(df["Input"][:10].values)

In [7]:
%store cite_abg_abs_cor_10

Stored 'cite_abg_abs_cor_10' (list)


In [8]:
%store

Stored variables and their in-db values:
abg_abs_cor_100                    -> ['ENSG00000129824_RPS4Y1', 'ENSG00000229807_XIST',
cite_abg_abs_cor_10                -> ['ENSG00000129824_RPS4Y1', 'ENSG00000229807_XIST',
cite_abg_abs_cor_100               -> ['ENSG00000129824_RPS4Y1', 'ENSG00000229807_XIST',
cite_abg_abs_cor_50                -> ['ENSG00000129824_RPS4Y1', 'ENSG00000229807_XIST',
cite_abg_abs_cor_500               -> ['ENSG00000129824_RPS4Y1', 'ENSG00000229807_XIST',
cite_avg_abs_corr_01               -> ['ENSG00000129824_RPS4Y1', 'ENSG00000229807_XIST',
cite_max_min_corr_01               -> ['ENSG00000129824_RPS4Y1', 'ENSG00000229807_XIST',
cite_max_min_corr_02               -> ['ENSG00000129824_RPS4Y1', 'ENSG00000229807_XIST',
cite_max_min_corr_03               -> ['ENSG00000129824_RPS4Y1', 'ENSG00000229807_XIST',
multi_abg_abs_cor_10               -> ['chr1:47180897-47181792', 'chr11:74011263-7401219
multi_abg_abs_cor_100              -> ['chr1:47180897-47181792', 'chr

In [9]:
%store -r

In [12]:
use = cite_abg_abs_cor_10

In [13]:
use

['ENSG00000129824_RPS4Y1',
 'ENSG00000229807_XIST',
 'ENSG00000198692_EIF1AY',
 'ENSG00000102145_GATA1',
 'ENSG00000128040_SPINK2',
 'ENSG00000198034_RPS4X',
 'ENSG00000105610_KLF1',
 'ENSG00000160789_LMNA',
 'ENSG00000131002_TXLNGY',
 'ENSG00000029534_ANK1']

In [15]:
use_name = "cite_abg_abs_cor_10"

In [16]:
columns_idx = 0
columns_idx_list = []
for i in train_X_idxcol["columns"]:
    if i in use:
        columns_idx_list.append(columns_idx)
    columns_idx += 1

In [17]:
columns_idx_list

[4731, 8579, 9734, 11575, 12290, 17308, 17320, 18729, 20449, 21097]

In [18]:
train_values = train_X_values[:, columns_idx_list]
test_values = test_X_values[:, columns_idx_list]

In [19]:
train_values.shape

(70988, 10)

In [20]:
test_values.shape

(48663, 10)

In [21]:
columns_idx = 0
columns_idx_list = []
for i in test_X_2_columns:
    if i in use:
        columns_idx_list.append(columns_idx)
    columns_idx += 1

In [21]:
columns_idx_list

[]

In [22]:
test2_values = test_X_2_values[:, columns_idx_list]

NameError: name 'test_X_2_values' is not defined

In [23]:
test2_values.shape

NameError: name 'test2_values' is not defined

In [22]:
scipy.sparse.save_npz(f"../../data/input/correlation/train_{use_name}_inputs.npz", scipy.sparse.csr_matrix(train_values))

In [23]:
scipy.sparse.save_npz(f"../../data/input/correlation/test_{use_name}_inputs.npz", scipy.sparse.csr_matrix(test_values))

In [26]:
scipy.sparse.save_npz(f"../../data/input/correlation/test2_{use_name}_inputs.npz", scipy.sparse.csr_matrix(test2_values))